# Uncertainties

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

from collections import defaultdict

import jax.numpy as jnp
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from IPython.display import Latex, Markdown
from matplotlib import cm
from tensorwaves.function import ParametrizedBackendFunction
from tensorwaves.function.sympy import create_parametrized_function
from tqdm.auto import tqdm

from polarization import formulate_polarization
from polarization.amplitude import AmplitudeModel
from polarization.data import create_data_transformer, generate_meshgrid_sample
from polarization.function import integrate_intensity, sub_intensity
from polarization.io import mute_jax_warnings, perform_cached_doit
from polarization.lhcb import (
    ParameterBootstrap,
    load_model_builder,
    load_model_parameters,
)
from polarization.lhcb.particle import load_particles
from polarization.plot import use_mpl_latex_fonts

mute_jax_warnings()
FUNCTION_CACHE: dict[sp.Expr, ParametrizedBackendFunction] = {}

## Model loading

In [ ]:
# fmt: off
allowed_model_titles = [
    "Default amplitude model",
    "Alternative amplitude model with K(892) with free mass and width",
    "Alternative amplitude model with L(1670) with free mass and width",
    "Alternative amplitude model with L(1690) with free mass and width",
    "Alternative amplitude model with D(1232) with free mass and width",
    "Alternative amplitude model with L(1600), D(1600), D(1700) with free mass and width",
    "Alternative amplitude model with K(700) with free mass and width",
    "Alternative amplitude model with K(1430) with free width",
    "Alternative amplitude model with free radial parameter d for the Lc resonance, indicated as dLc",
]
# fmt: on

In [ ]:
model_file = "../data/model-definitions.yaml"
particles = load_particles("../data/particle-definitions.yaml")
reference_subsystem = 1

models = {}
progress_bar = tqdm(desc="Formulating models", total=len(allowed_model_titles))
for title in allowed_model_titles:
    progress_bar.set_postfix_str(title)
    amplitude_builder = load_model_builder(model_file, particles, model_id=title)
    model = amplitude_builder.formulate(reference_subsystem)
    imported_parameter_values = load_model_parameters(model_file, model.decay, title)
    model.parameter_defaults.update(imported_parameter_values)
    models[title] = model
    progress_bar.update()
progress_bar.set_postfix_str("")
progress_bar.close()

In [ ]:
unfolded_exprs = {}
progress_bar = tqdm(desc="Unfolding expressions", total=len(models))
for title, model in models.items():
    progress_bar.set_postfix_str(title)
    polarization_exprs = formulate_polarization(
        amplitude_builder, reference_subsystem
    )
    unfolded_exprs[title] = {
        f"alpha_{i}": perform_cached_doit(expr.doit().xreplace(model.amplitudes))
        for i, expr in zip("xyz", polarization_exprs)
    }
    unfolded_exprs[title]["intensity"] = perform_cached_doit(model.full_expression)
    progress_bar.update()
progress_bar.set_postfix_str("")
progress_bar.close()

In [ ]:
unique_model_hashes = {hash(exprs["intensity"]) for exprs in unfolded_exprs.values()}
src = f"""
Of the {len(allowed_model_titles)} imported models, there are
{len(unique_model_hashes)} with a unique expression tree.
"""
Markdown(src)

## Numerical function creation

This time, we do not {ref}`substitute certain parameters with their parameter defaults<polarization:Definition of free parameters>`, but lambdify the full expression, so that parameter values can be set for different models. Note that this makes lambdification slower. This is mitigated by caching the lambdified function if using the unfolded expression as cache key.

In [ ]:
def cached_lambdify(
    expr: sp.Expr, model: AmplitudeModel
) -> ParametrizedBackendFunction:
    func = FUNCTION_CACHE.get(expr)
    if func is None:
        func = create_parametrized_function(
            expr,
            parameters=model.parameter_defaults,
            backend="jax",
        )
        FUNCTION_CACHE[expr] = func
    str_parameters = {str(k): v for k, v in model.parameter_defaults.items()}
    func.update_parameters(str_parameters)
    return func


jax_functions = {}
original_parameters: dict[str, dict[str, complex | float | int]] = {}
progress_bar = tqdm(desc="Lambdifying to JAX", total=len(models))
for title, model in models.items():
    progress_bar.set_postfix_str(title)
    jax_functions[title] = {
        key: cached_lambdify(expr, model)
        for key, expr in tqdm(unfolded_exprs[title].items(), leave=False)
    }
    original_parameters[title] = dict(jax_functions[title]["intensity"].parameters)
    progress_bar.update()
progress_bar.set_postfix_str("")
progress_bar.close()

## Statistical uncertainties

### Parameter bootstrapping

In [ ]:
n_bootstraps = 200
nominal_model_title: str = "Default amplitude model"
nominal_functions = jax_functions[nominal_model_title]
bootstrap = ParameterBootstrap(model_file, model.decay, nominal_model_title)
bootstrap_parameters = bootstrap.create_distribution(n_bootstraps, seed=0)

In [ ]:
resolution = 200
transformer = create_data_transformer(model)
phsp = generate_meshgrid_sample(model.decay, resolution)
data = transformer(phsp)
X = data["sigma1"]
Y = data["sigma2"]

In [ ]:
resonances = [chain.resonance for chain in model.decay.chains]
nominal_parameters = dict(original_parameters[nominal_model_title])
stat_grids = defaultdict(list)
stat_decay_rates = defaultdict(list)
for i in tqdm(
    range(n_bootstraps),
    desc="Computing polarizations and intensities for parameter combinations",
):
    new_parameters = {k: v[i] for k, v in bootstrap_parameters.items()}
    for key, func in nominal_functions.items():
        func.update_parameters(nominal_parameters)
        func.update_parameters(new_parameters)
        stat_grids[key].append(func(data).real)
    I_tot = integrate_intensity(stat_grids["intensity"][-1])
    for resonance in resonances:
        res_filter = resonance.name.replace("(", R"\(").replace(")", R"\)")
        I_sub = sub_intensity(nominal_functions["intensity"], data, [res_filter])
        stat_decay_rates[resonance.name].append(I_sub / I_tot)

In [ ]:
stat_intensities = jnp.array(stat_grids["intensity"])
stat_polarizations = jnp.array([stat_grids[f"alpha_{i}"] for i in "xyz"])
stat_polarization_norms = jnp.sqrt(jnp.sum(stat_polarizations**2, axis=0))
stat_decay_rates = {k: jnp.array(v) for k, v in stat_decay_rates.items()}

### Mean and standard deviations

In [ ]:
assert stat_intensities.shape == (n_bootstraps, resolution, resolution)
assert stat_polarizations.shape == (3, n_bootstraps, resolution, resolution)
assert stat_polarization_norms.shape == (n_bootstraps, resolution, resolution)
(n_bootstraps, resolution, resolution)

In [ ]:
stat_alpha_mean = [
    jnp.mean(stat_polarization_norms, axis=0),
    *jnp.mean(stat_polarizations, axis=1),
]
stat_alpha_std = [
    jnp.std(stat_polarization_norms, axis=0),
    *jnp.std(stat_polarizations, axis=1),
]

stat_alpha_times_I_mean = [
    jnp.mean(stat_polarization_norms * stat_intensities, axis=0),
    *jnp.mean(stat_polarizations * stat_intensities, axis=1),
]
stat_alpha_times_I_std = [
    jnp.std(stat_polarization_norms * stat_intensities, axis=0),
    *jnp.std(stat_polarizations * stat_intensities, axis=1),
]
stat_alpha_times_I_mean = jnp.array(stat_alpha_times_I_mean)
stat_alpha_times_I_std = jnp.array(stat_alpha_times_I_std)

stat_intensity_mean = jnp.mean(stat_intensities, axis=0)
stat_intensity_std = jnp.std(stat_intensities, axis=0)

### Distributions

In [ ]:
%config InlineBackend.figure_formats = ['png']

In [ ]:
plt.rcdefaults()
use_mpl_latex_fonts()
plt.rc("font", size=18)
fig, axes = plt.subplots(
    dpi=200,
    figsize=(16.7, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.18]},
    ncols=4,
    nrows=2,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(hspace=0.02, wspace=0.02)
fig.suptitle(R"Polarization sensitivity $\vec\alpha$ (statistical)")
s1_label = R"$m^2\left(K\pi\right)$ [GeV$/c^2$]"
s2_label = R"$m^2\left(pK\right)$ [GeV$/c^2$]"
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

global_max_std = max(map(jnp.nanmax, stat_alpha_std))
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)

    Z = stat_alpha_mean[i]
    mesh = axes[0, i].pcolormesh(X, Y, Z, cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-1, vmax=+1)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha$ averaged with {n_bootstraps} bootstraps")
        c_bar.ax.set_yticks([-1, 0, +1])
        c_bar.ax.set_yticklabels(["-1", "0", "+1"])

    Z = stat_alpha_std[i]
    mesh = axes[1, i].pcolormesh(X, Y, Z)
    mesh.set_clim(vmin=0, vmax=global_max_std)
    axes[1, i].set_xlabel(s1_label)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation")
plt.show()

In [ ]:
fig, axes = plt.subplots(
    dpi=200,
    figsize=(16.7, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.18]},
    ncols=4,
    nrows=2,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(hspace=0.02, wspace=0.02)
fig.suptitle(R"$\vec\alpha \cdot I$ distributions (statistical)")
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

global_max_mean = jnp.nanmax(jnp.abs(stat_alpha_times_I_mean))
global_max_std = jnp.nanmax(stat_alpha_times_I_std / stat_intensity_mean)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)
    axes[1, i].set_xlabel(s1_label)

    Z = stat_alpha_times_I_mean[i]
    mesh = axes[0, i].pcolormesh(X, Y, Z, cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-global_max_mean, vmax=+global_max_mean)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(
            Rf"$\alpha \cdot I$ averaged with {n_bootstraps} bootstraps"
        )

    Z = stat_alpha_times_I_std[i] / stat_intensity_mean
    mesh = axes[1, i].pcolormesh(X, Y, Z)
    mesh.set_clim(vmin=0, vmax=global_max_std)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("standard deviation / intensity")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    dpi=200,
    figsize=(12, 6.2),
    ncols=2,
    sharey=True,
)
fig.suptitle("Intensity distribution (statistical)", y=0.95)
ax1.set_xlabel(s1_label)
ax2.set_xlabel(s1_label)
ax1.set_ylabel(s2_label)

Z = stat_intensity_mean
mesh = ax1.pcolormesh(X, Y, Z, cmap=cm.Reds)
fig.colorbar(mesh, ax=ax1, pad=0.01)
ax1.set_title(f"average of {n_bootstraps} bootstraps")

Z = stat_intensity_std / stat_intensity_mean
mesh = ax2.pcolormesh(X, Y, Z)
fig.colorbar(mesh, ax=ax2, pad=0.01)
ax2.set_title("standard deviation / intensity")
fig.tight_layout()
plt.show()

### Comparison with nominal values

In [ ]:
for func in nominal_functions.values():
    func.update_parameters(nominal_parameters)
nominal_intensity = nominal_functions["intensity"](data)
nominal_polarizations = jnp.array(
    [nominal_functions[f"alpha_{i}"](data).real for i in "xyz"]
)
nominal_polarization_norms = jnp.sqrt(jnp.sum(nominal_polarizations**2, axis=0))

In [ ]:
fig, axes = plt.subplots(
    dpi=200,
    figsize=(17.3, 4),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.2]},
    ncols=4,
    sharey=True,
)
plt.subplots_adjust(hspace=0.2, wspace=0.05)
fig.suptitle("Comparison with nominal values", y=1.04)
axes[0].set_ylabel(s2_label)
for ax in axes:
    ax.set_xlabel(s1_label)

vmax = 5.0  # %
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
        z_values = jnp.abs(
            (stat_alpha_mean[i] - nominal_polarizations[i - 1])
            / nominal_polarizations[i - 1]
        )
    else:
        title = "$I$"
        z_values = 100 * jnp.abs(
            (stat_intensity_mean - nominal_intensity) / nominal_intensity
        )
    axes[i].set_title(title)

    Z = z_values
    mesh = axes[i].pcolormesh(X, Y, Z, cmap=cm.Reds)
    mesh.set_clim(vmin=0, vmax=vmax)
    if axes[i] is axes[-1]:
        c_bar = fig.colorbar(mesh, ax=axes[i], pad=0.02)
        c_bar.ax.set_ylabel(R"difference with nominal value (\%)")

plt.show()

## Systematic uncertainties

In [ ]:
syst_grids = defaultdict(list)
syst_decay_rates = defaultdict(list)
progress_bar = tqdm(desc="Computing systematics", total=len(models))
for title, model in models.items():
    progress_bar.set_postfix_str(title)
    for key, func in jax_functions[title].items():
        func.update_parameters(original_parameters[title])
        syst_grids[key].append(func(data).real)
    I_tot = integrate_intensity(syst_grids["intensity"][-1])
    intensity_func = jax_functions[title]["intensity"]
    for resonance in resonances:
        res_filter = resonance.name.replace("(", R"\(").replace(")", R"\)")
        I_sub = sub_intensity(intensity_func, data, [res_filter])
        syst_decay_rates[resonance.name].append(I_sub / I_tot)
    progress_bar.update()
progress_bar.set_postfix_str("")
progress_bar.close()

In [ ]:
syst_intensities = jnp.array(syst_grids["intensity"])
syst_polarizations = jnp.array([syst_grids[f"alpha_{i}"] for i in "xyz"])
syst_polarization_norms = jnp.sqrt(jnp.sum(syst_polarizations**2, axis=0))
syst_decay_rates = {k: jnp.array(v) for k, v in syst_decay_rates.items()}

### Mean and standard deviations

In [ ]:
n_models = len(models)
assert syst_intensities.shape == (n_models, resolution, resolution)
assert syst_polarizations.shape == (3, n_models, resolution, resolution)
assert syst_polarization_norms.shape == (n_models, resolution, resolution)
n_models, resolution, resolution

In [ ]:
syst_alpha_mean = [
    jnp.mean(syst_polarization_norms, axis=0),
    *jnp.mean(syst_polarizations, axis=1),
]
alpha_diff_with_model_0 = [
    syst_polarization_norms - syst_polarization_norms[0],
    *(syst_polarizations - syst_polarizations[:, None, 0]),
]

syst_alpha_mean = jnp.array(syst_alpha_mean)
alpha_diff_with_model_0 = jnp.array(alpha_diff_with_model_0)

assert alpha_diff_with_model_0.shape == (4, n_models, resolution, resolution)
assert jnp.nanmax(alpha_diff_with_model_0[:, 0]) == 0.0
alpha_syst_extrema = jnp.abs(alpha_diff_with_model_0).max(axis=1)

In [ ]:
syst_polarizations_times_I = [
    syst_polarization_norms * syst_intensities,
    *(syst_polarizations * syst_intensities),
]
syst_polarizations_times_I = jnp.array(syst_polarizations_times_I)


syst_alpha_times_I_mean = syst_polarizations_times_I.mean(axis=1)
syst_alpha_times_I_diff = (
    syst_polarizations_times_I - syst_polarizations_times_I[:, None, 0]
)
assert syst_alpha_times_I_diff.shape == (4, n_models, resolution, resolution)
assert jnp.nanmax(syst_alpha_times_I_diff[:, 0]) == 0.0
syst_alpha_times_I_extrema = jnp.abs(syst_alpha_times_I_diff).max(axis=1)

In [ ]:
intensity_diff_with_model_0 = syst_intensities - syst_intensities[0]
intensity_extrema = jnp.nanmax(intensity_diff_with_model_0, axis=0)

### Distributions

In [ ]:
fig, axes = plt.subplots(
    dpi=200,
    figsize=(16.7, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.18]},
    ncols=4,
    nrows=2,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(hspace=0.02, wspace=0.02)
fig.suptitle(R"Polarization sensitivity $\vec\alpha$ (systematics)")
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

global_max_std = jnp.nanmax(alpha_syst_extrema)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)

    Z = syst_alpha_mean[i]
    mesh = axes[0, i].pcolormesh(X, Y, Z, cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-1, vmax=+1)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha$ averaged with {n_models} models")
        c_bar.ax.set_yticks([-1, 0, +1])
        c_bar.ax.set_yticklabels(["-1", "0", "+1"])

    Z = alpha_syst_extrema[i]
    mesh = axes[1, i].pcolormesh(X, Y, Z)
    mesh.set_clim(vmin=0, vmax=global_max_std)
    axes[1, i].set_xlabel(s1_label)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel("maximum absolute difference\nwith the default model")
plt.show()

In [ ]:
fig, axes = plt.subplots(
    dpi=200,
    figsize=(16.7, 8),
    gridspec_kw={"width_ratios": [1, 1, 1, 1.18]},
    ncols=4,
    nrows=2,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(hspace=0.02, wspace=0.02)
axes[0, 0].set_ylabel(s2_label)
axes[1, 0].set_ylabel(s2_label)

syst_intensity_mean = jnp.mean(syst_intensities, axis=0)
global_max_mean = jnp.nanmax(jnp.abs(syst_alpha_times_I_mean))
global_max_diff = jnp.nanmax(syst_alpha_times_I_extrema / syst_intensity_mean)
for i in range(4):
    if i != 0:
        title = Rf"$\alpha_{'xyz'[i-1]}$"
    else:
        title = R"$\left|\vec\alpha\right|$"
    axes[0, i].set_title(title)
    axes[1, i].set_xlabel(s1_label)

    Z = syst_alpha_times_I_mean[i]
    mesh = axes[0, i].pcolormesh(X, Y, Z, cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-global_max_mean, vmax=+global_max_mean)
    if axes[0, i] is axes[0, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[0, i], pad=0.01)
        c_bar.ax.set_ylabel(Rf"$\alpha \cdot I$ averaged with {n_models} models")

    Z = syst_alpha_times_I_extrema[i] / syst_intensity_mean
    mesh = axes[1, i].pcolormesh(X, Y, Z)
    mesh.set_clim(vmin=0, vmax=global_max_diff)
    if axes[1, i] is axes[1, -1]:
        c_bar = fig.colorbar(mesh, ax=axes[1, i], pad=0.01)
        c_bar.ax.set_ylabel(
            "maximum absolute difference\n"
            "with the default model\n"
            "divided by nominal intensity"
        )
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    dpi=200,
    figsize=(12, 6.9),
    ncols=2,
    sharey=True,
)
fig.suptitle("Intensity distribution (systematics)", y=0.95)
ax1.set_xlabel(s1_label)
ax2.set_xlabel(s1_label)
ax1.set_ylabel(s2_label)

Z = syst_intensity_mean
mesh = ax1.pcolormesh(X, Y, Z, cmap=cm.Reds)
fig.colorbar(mesh, ax=ax1, pad=0.01)
ax1.set_title(f"average of {n_models} models")

Z = intensity_extrema / syst_intensity_mean
mesh = ax2.pcolormesh(X, Y, Z)
fig.colorbar(mesh, ax=ax2, pad=0.01)
ax2.set_title("maximum absolute difference\n" R"with the default model (\%)")
fig.tight_layout()
plt.show()

## Fit fractions

In [ ]:
src = r"""
| Resonance | Fit Fraction (%) |
|----------:|:-----------------|
"""
for resonance in resonances:
    ff_statistical = 100 * stat_decay_rates[resonance.name]
    ff_systematics = 100 * syst_decay_rates[resonance.name]
    ff_nominal = f"{ff_systematics[0]:.2f}"
    ff_stat = f"{ff_statistical.std():.2f}"
    ff_syst_min = f"{(ff_systematics[1:]-ff_systematics[0]).min():+.2f}"
    ff_syst_max = f"{(ff_systematics[1:]-ff_systematics[0]).max():+.2f}"
    src += f"| ${resonance.latex}$ | "
    src += (
        Rf"${ff_nominal} \pm {ff_stat}_{{{ff_syst_min}}}^{{{ff_syst_max}}}$ |" "\n"
    )
Markdown(src)

In [ ]:
src = "|   |"
for i, _ in enumerate(models):
    if i == 0:
        continue
    src += f" {i} |"
src += "\n"
for _ in models:
    src += "|:---:"
src += "| \n"

for resonance in resonances:
    ff_systematics = 100 * syst_decay_rates[resonance.name]
    src += f"| ${resonance.latex}$ | "
    for ff_model in ff_systematics[1:]:
        diff = f"{ff_model-ff_systematics[0]:+.2f}"
        if ff_model == ff_systematics[1:].min():
            src += f'<span style="color:blue">{diff}</span> | '
        elif ff_model == ff_systematics[1:].max():
            src += f'<span style="color:red">{diff}</span> | '
        else:
            src += f"{diff} | "
    src += "\n"

for i, title in enumerate(models):
    src += f"\n- **{i}**: {title}"
Markdown(src)

## Average polarizations

In [ ]:
def compute_weighted_average(v: jnp.ndarray, weights: jnp.ndarray) -> jnp.ndarray:
    return jnp.nansum(v * weights, axis=(-1, -2)) / jnp.nansum(
        weights, axis=(-1, -2)
    )


stat_weighted_alpha_norm = compute_weighted_average(
    stat_polarization_norms,
    weights=stat_intensities,
)
stat_weighted_alpha = compute_weighted_average(
    stat_polarizations,
    weights=stat_intensities,
)

syst_weighted_alpha_norm = compute_weighted_average(
    syst_polarization_norms,
    weights=syst_intensities,
)
syst_weighted_alpha = compute_weighted_average(
    syst_polarizations,
    weights=syst_intensities,
)

nominal_weighted_alpha_norm = syst_weighted_alpha_norm[0]
nominal_weighted_alpha = syst_weighted_alpha[:, 0]

In [ ]:
syst_weighted_alpha_norm_diff = (
    syst_weighted_alpha_norm - syst_weighted_alpha_norm[0]
)
syst_weighted_alpha_diff = (syst_weighted_alpha.T - syst_weighted_alpha[:, 0]).T

stat_weighted_alpha_std = stat_weighted_alpha.std(axis=1)
syst_weighted_alpha_min = syst_weighted_alpha_diff.min(axis=1)
syst_weighted_alpha_max = syst_weighted_alpha_diff.max(axis=1)

In [ ]:
def render_uncertainties(value, stat, syst_min, syst_max, plus: bool = True) -> str:
    if plus:
        val = f"{1e3*value:+.1f}"
    else:
        val = f"{1e3*value:.1f}"
    stat = f"{1e3*stat:.1f}"
    syst_min = f"-{1e3*abs(syst_min):.1f}"
    syst_max = f"+{1e3*abs(syst_max):.1f}"
    return (
        Rf"\left({val} \pm {stat}_{{{syst_min}}}^{{{syst_max}}} \right) \times"
        " 10^{-3}"
    )


src = R"\begin{array}{ccr}"
for i in range(4):
    if i < 3:
        title = Rf"\alpha_{'xyz'[i]}"
        value_with_uncertainties = render_uncertainties(
            nominal_weighted_alpha[i],
            stat_weighted_alpha_std[i],
            syst_weighted_alpha_min[i],
            syst_weighted_alpha_max[i],
        )
    else:
        title = R"\left|\vec{{\alpha}}\right|"
        value_with_uncertainties = render_uncertainties(
            nominal_weighted_alpha_norm,
            stat_weighted_alpha_norm.std(),
            syst_weighted_alpha_norm_diff.min(),
            syst_weighted_alpha_norm_diff.max(),
            plus=False,
        )
    src += Rf"\overline{{{title}}} & = & {value_with_uncertainties} \\"
src += R"\end{array}"
Latex(src)

Difference between each model and nominal value (computed with the default model):

In [ ]:
pd.DataFrame(
    {
        title: (f"{x:+.4f}", f"{y:+.4f}", f"{z:+.4f}", f"{norm:+.4f}")
        for title, x, y, z, norm in zip(
            models,
            *syst_weighted_alpha_diff,
            syst_weighted_alpha_norm_diff,
        )
    }
).transpose().rename(columns={0: "ɑx", 1: "ɑy", 2: "ɑz", 3: "|ɑ|"})